In [13]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


25 documents
7 classes ['formality', 'ghost', 'goodbye', 'greeting', 'name', 'thanks', 'work']
41 unique lemmatized words ["'s", ',', 'anyone', 'are', 'awesome', 'believe', 'bye', 'chatting', 'day', 'doing', 'existence', 'for', 'ghost', 'good', 'goodbye', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'in', 'is', 'later', 'me', 'name', 'next', 'nice', 'of', 'see', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'you', 'your']
Training data created
Epoch 1/200
25/25 [==============================] - 1s 24ms/step - loss: 1.8852 - accuracy: 0.2400
Epoch 2/200
25/25 [==============================] - 0s 480us/step - loss: 1.9019 - accuracy: 0.2800
Epoch 3/200
25/25 [==============================] - 0s 1ms/step - loss: 1.8951 - accuracy: 0.2800
Epoch 4/200
25/25 [==============================] - 0s 639us/step - loss: 1.8656 - accuracy: 0.2800
Epoch 5/200
25/25 [==============================] - 0s 640us/step - loss: 1.8275 - accuracy: 0.2000
Epoch 6/200
25/2

25/25 [==============================] - 0s 640us/step - loss: 0.2244 - accuracy: 0.9200
Epoch 77/200
25/25 [==============================] - 0s 640us/step - loss: 0.1919 - accuracy: 0.8800
Epoch 78/200
25/25 [==============================] - 0s 800us/step - loss: 0.2674 - accuracy: 0.8800
Epoch 79/200
25/25 [==============================] - 0s 960us/step - loss: 0.1201 - accuracy: 0.9600
Epoch 80/200
25/25 [==============================] - 0s 960us/step - loss: 0.2017 - accuracy: 0.9600
Epoch 81/200
25/25 [==============================] - 0s 961us/step - loss: 0.1413 - accuracy: 0.9200
Epoch 82/200
25/25 [==============================] - 0s 960us/step - loss: 0.2243 - accuracy: 0.8800
Epoch 83/200
25/25 [==============================] - 0s 960us/step - loss: 0.1769 - accuracy: 0.9600
Epoch 84/200
25/25 [==============================] - 0s 960us/step - loss: 0.1507 - accuracy: 0.9200
Epoch 85/200
25/25 [==============================] - 0s 640us/step - loss: 0.2274 - accuracy: 

25/25 [==============================] - 0s 960us/step - loss: 0.1758 - accuracy: 0.9200
Epoch 155/200
25/25 [==============================] - 0s 960us/step - loss: 0.0718 - accuracy: 1.0000
Epoch 156/200
25/25 [==============================] - 0s 480us/step - loss: 0.1900 - accuracy: 0.9600
Epoch 157/200
25/25 [==============================] - 0s 800us/step - loss: 0.0907 - accuracy: 0.9600
Epoch 158/200
25/25 [==============================] - 0s 961us/step - loss: 0.1349 - accuracy: 0.9200
Epoch 159/200
25/25 [==============================] - 0s 960us/step - loss: 0.0966 - accuracy: 0.9600
Epoch 160/200
25/25 [==============================] - 0s 801us/step - loss: 0.2923 - accuracy: 0.9200
Epoch 161/200
25/25 [==============================] - 0s 800us/step - loss: 0.0933 - accuracy: 0.9600
Epoch 162/200
25/25 [==============================] - 0s 800us/step - loss: 0.0661 - accuracy: 1.0000
Epoch 163/200
25/25 [==============================] - 0s 800us/step - loss: 0.0966 - a

In [28]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


ChatLog = Text(base, bd=0, bg="cyan", height="8", width="50", font="Arial")

ChatLog.config(state=DISABLED)


scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set


SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

EntryBox = Text(base, bd=0, bg="",width="29", height="5", font="Arial",fg="#000000")

scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=6, y=401, height=90, width=265)
SendButton.place(x=266, y=401, height=90)

base.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
